In [1]:
'''
Using two popular live video processing libraries known as 
Mediapipe and Open-CV, we can take webcam input and run our 
previously developed model on real time video stream.
'''
from keras.models import load_model
import mediapipe as mp
import numpy as np
import pandas as pd
import time
import cv2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


In [2]:
'''
define some variables such as the saved model and 
information on the camera for Open-CV.Each of the variables 
set here are grouped into one of four categories. The first 
category pertains directly to the model. The second and third sections of the 
code define variables required to run and start Mediapipe 
and Open-CV. The final category is used primarily to 
analyze the frame when detected, and create the dictionary 
used in the cross-referencing of the data provided by the 
image model.
'''
model = load_model('smnist.h5')
mphands = mp.solutions.hands
hands = mphands.Hands()
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)
_, frame = cap.read()
h, w, c = frame.shape
img_counter = 0
analysisframe = ''
letterpred = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']



This section of the program takes the input from your camera and uses our imported image processing library to display the input from the device to the computer. This portion of code focuses on getting general information from your camera and simply showing it back in a new window. However, using the Mediapipe library, we can detect the major landmarks of the hand such as the fingers and palms, and create a bounding box around the hand.  
  
The idea of a bounding box is a crucial component to all forms of image classification and analysis. The box allows the model to focus directly on the portion of the image needed for the function. Without this, the algorithm finds patterns in wrong places and can cause an incorrect result.  
  
Now , we make use of the image reshaping feature from Open-CV to resize the image to the dimensions of the bounding box, rather than creating a visual object around it. Along with this, we also use NumPy and Open-CV to modify the image to be the same dimensions as the images the model was trained on.  
  
Towards the top of the code, you may notice the odd sequence of variables being defined. This is due to the nature of the camera library syntax. When an image is processed and changed by Open-CV, the changes are made on top of the frame used, essentially saving the changes made to the image. The definition of multiple variables of equal value makes it so that the frame displayed in by the function is separate from the picture on which the model is being ran on.  
  
Finally, we need to run the trained model on the processed image and process the information output.

In [3]:
while True:
    _, frame = cap.read()

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        analysisframe = frame
        showframe = analysisframe
        cv2.imshow("Frame", showframe)
        framergbanalysis = cv2.cvtColor(analysisframe, cv2.COLOR_BGR2RGB)
        resultanalysis = hands.process(framergbanalysis)
        hand_landmarksanalysis = resultanalysis.multi_hand_landmarks
        if hand_landmarksanalysis:
            for handLMsanalysis in hand_landmarksanalysis:
                x_max = 0
                y_max = 0
                x_min = w
                y_min = h
                for lmanalysis in handLMsanalysis.landmark:
                    x, y = int(lmanalysis.x * w), int(lmanalysis.y * h)
                    if x > x_max:
                        x_max = x
                    if x < x_min:
                        x_min = x
                    if y > y_max:
                        y_max = y
                    if y < y_min:
                        y_min = y
                y_min -= 20
                y_max += 20
                x_min -= 20
                x_max += 20 

        analysisframe = cv2.cvtColor(analysisframe, cv2.COLOR_BGR2GRAY)
        analysisframe = analysisframe[y_min:y_max, x_min:x_max]
        analysisframe = cv2.resize(analysisframe,(28,28))


        nlist = []
        rows,cols = analysisframe.shape
        for i in range(rows):
            for j in range(cols):
                k = analysisframe[i,j]
                nlist.append(k)
        
        datan = pd.DataFrame(nlist).T
        colname = []
        for val in range(784):
            colname.append(val)
        datan.columns = colname

        pixeldata = datan.values
        pixeldata = pixeldata / 255
        pixeldata = pixeldata.reshape(-1,28,28,1)

        '''
        The first two lines draw the predicted probabilities that a hand image is any of the different classes from Keras. The data is presented in the form of 2 tensors, of which, the first tensor contains information on the probabilities. A tensor is essentially a collection of feature vectors, very similar to an array. The tensor produced by the model is one dimensional, allowing it to be used with the linear algebra library NumPy to parse the information into a more pythonic form.

        From here, we utilize the previously created list of classes under the variable letterpred to create a dictionary, matching the values from the tensor to the keys. This allows us to match each letter’s probability with the class it corresponds to.

        Following this step, we use list comprehension to order and sort the values from highest to lowest. This then allows us to take the first few items in the list and designate them the 3 letters that closest correspond to the Sign Language image shown.

        Finally, we use a for loop to cycle through all of the key:value pairs in the dictionary created to match the highest values to their corresponding keys and print out the output with each letter’s probability.
        '''
        prediction = model.predict(pixeldata)
        predarray = np.array(prediction[0])
        letter_prediction_dict = {letterpred[i]: predarray[i] for i in range(len(letterpred))}
        predarrayordered = sorted(predarray, reverse=True)
        high1 = predarrayordered[0]
        high2 = predarrayordered[1]
        high3 = predarrayordered[2]
        for key,value in letter_prediction_dict.items():
            if value==high1:
                print("Predicted Character 1: ", key)
                print('Confidence 1: ', 100*value)
            elif value==high2:
                print("Predicted Character 2: ", key)
                print('Confidence 2: ', 100*value)
            elif value==high3:
                print("Predicted Character 3: ", key)
                print('Confidence 3: ', 100*value)
        time.sleep(5)

    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(framergb)
    hand_landmarks = result.multi_hand_landmarks
    if hand_landmarks:
        for handLMs in hand_landmarks:
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h
            for lm in handLMs.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y
            y_min -= 20
            y_max += 20
            x_min -= 20
            x_max += 20
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    cv2.imshow("Frame", frame)

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 517ms/step
Predicted Character 2:  H
Confidence 2:  6.596560776233673
Predicted Character 1:  P
Confidence 1:  91.59116744995117
Predicted Character 3:  Y
Confidence 3:  1.718909665942192
1/1 [==============================] - 0s 20ms/step
Predicted Character 3:  N
Confidence 3:  17.279380559921265
Predicted Character 1:  P
Confidence 1:  45.84163725376129
Predicted Character 2:  T
Confidence 2:  23.513858020305634
Escape hit, closing...
